# Entregável 3 de Visão Computacional e Robótica


Entrega até 12/03 ao fim do atendimento

Pode ser feito **em duplas**

Nesta semana vamos trabalhar com um assunto extremamente atual: reconhecimento de objetos e rastreamento


Referências:

[https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/](https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/)

[https://github.com/iArunava/YOLOv3-Object-Detection-with-OpenCV/](https://github.com/iArunava/YOLOv3-Object-Detection-with-OpenCV/)

[https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/](https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/)

Ouça a explicacão do professor sobre rastreamento e deteção

## Alunos: Lucas Muchaluat e Andre Weber

### 1. Executar os três exemplos

Há três exemplos: `mobilenet_detection`, `yolov3_detection` e `tracking`.

Os dois primeiros são reconhecedores de objetos, e o último é de rastreamento


Um dos arquivos abaixo precisa ser baixado e salvo nas pasta  `yolov3_detection/yolov3-coco` .

[https://www.dropbox.com/s/013ogt2bhwfzxwb/yolov3.weights?dl=0](https://www.dropbox.com/s/013ogt2bhwfzxwb/yolov3.weights?dl=0) ou [https://pjreddie.com/media/files/yolov3.weights](https://pjreddie.com/media/files/yolov3.weights)

## 2. Identificar objeto

Escolha um dos projetos: `yolov3_detection` ou `mobilenet_detection` para basear seu código. 

Neste projeto, escolha uma categoria de objetos que o reconhecedor reconhece. Diga aqui qual foi sua escolha

Escolha: CAR

## 3. Rastrear

Implemente a seguinte funcionalidade: sempre que o objeto identificado em (2) estiver presente por mais que 5 frames seguidos, pare de rodar a identificação e comece a rastreá-lo.

Faça um vídeo com a  demonstração funcionalidade e mostre o link ao professor

Link: https://youtu.be/UlR5qervZN4

### Identificando objeto (carro) e trackeando

In [ ]:
# Para RODAR
# python object_detection_webcam.py --prototxt MobileNetSSD_deploy.prototxt.txt --model MobileNetSSD_deploy.caffemodel
# Credits: https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/

print("Para executar:\npython object_detection_webcam.py --prototxt MobileNetSSD_deploy.prototxt.txt --model MobileNetSSD_deploy.caffemodel")

# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import imutils
import time
import numpy as np
import argparse
import cv2

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--prototxt", required=True,
	help="path to Caffe 'deploy' prototxt file")
ap.add_argument("-m", "--model", required=True,
	help="path to Caffe pre-trained model")
ap.add_argument("-c", "--confidence", type=float, default=0.2,
	help="minimum probability to filter weak detections")
ap.add_argument("-v", "--video", type=str,
    help="path to input video file")
ap.add_argument("-t", "--tracker", type=str, default="kcf",
    help="OpenCV object tracker type")
args = vars(ap.parse_args())

# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

# extract the OpenCV version info
(major, minor) = cv2.__version__.split(".")[:2]

# if we are using OpenCV 3.2 OR BEFORE, we can use a special factory
# function to create our object tracker
if int(major) == 3 and int(minor) < 3:
    tracker = cv2.Tracker_create(args["tracker"].upper())

# otherwise, for OpenCV 3.3 OR NEWER, we need to explicity call the
# approrpiate object tracker constructor:
else:
    # initialize a dictionary that maps strings to their corresponding
    # OpenCV object tracker implementations
    OPENCV_OBJECT_TRACKERS = {
        "csrt": cv2.TrackerCSRT_create,
        "kcf": cv2.TrackerKCF_create,
        "boosting": cv2.TrackerBoosting_create,
        "mil": cv2.TrackerMIL_create,
        "tld": cv2.TrackerTLD_create,
        "medianflow": cv2.TrackerMedianFlow_create,
        "mosse": cv2.TrackerMOSSE_create
    }

    # grab the appropriate object tracker using our dictionary of
    # OpenCV object tracker objects
    tracker = OPENCV_OBJECT_TRACKERS[args["tracker"]]()



# initialize the bounding box coordinates of the object we are going
# to track
initBB = None

# initialize the FPS throughput estimator
fps = None


# load the input image and construct an input blob for the image
# by resizing to a fixed 300x300 pixels and then normalizing it
# (note: normalization is done via the authors of the MobileNet SSD
# implementation)


def detect(frame):
    image = frame.copy()
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()

    results = []

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence


        if confidence > args["confidence"]:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

            results.append((CLASSES[idx], confidence*100, (startX, startY),(endX, endY) ))

    # show the output image
    return image, results






import cv2

#cap = cv2.VideoCapture('hall_box_battery_1024.mp4')
cap = cv2.VideoCapture(0)

print("Known classes")
print(CLASSES)

count_frame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # resize the frame (so we can process it faster) and grab the frame dimensions
    frame = imutils.resize(frame, width=500)
    (H, W) = frame.shape[:2]

    if count_frame < 5:
        result_frame, result_tuples = detect(frame)
        if len(result_tuples)>0:
            if result_tuples[0][0] == "car":
                count_frame += 1
            else:
                count_frame = 0
    else:
        x1 =  result_tuples[0][2][0]
        y1 = result_tuples[0][2][1]
        x2 = result_tuples[0][3][0]
        y2 = result_tuples[0][3][1]
        difx = x2 - x1
        dify = y2 - y1

        initBB = (x1, y1, abs(difx), abs(dify))

        # start OpenCV object tracker using the supplied bounding box
        # coordinates, then start the FPS throughput estimator as well
        tracker.init(frame, initBB)
        fps = FPS().start()

        # check to see if we are currently tracking an object
        if initBB is not None:
            # grab the new bounding box coordinates of the object
            (success, box) = tracker.update(frame)

            # check to see if the tracking was a success
            if success:
                (x, y, w, h) = [int(v) for v in box]
                cv2.rectangle(frame, (x, y), (x + w, y + h),
                    (0, 255, 0), 2)
            # update the FPS counter
            fps.update()
            fps.stop()

            # initialize the set of information we'll be displaying on
            # the frame
            info = [
                ("Tracker", args["tracker"]),
                ("Success", "Yes" if success else "No"),
                ("FPS", "{:.2f}".format(fps.fps())),
            ]

            # loop over the info tuples and draw them on our frame
            for (i, (k, v)) in enumerate(info):
                text = "{}: {}".format(k, v)
                cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('frame',frame)

    # Prints the structures results:
    # Format:w
    # ("CLASS", confidence, (x1, y1, x2, y3))
    for t in result_tuples:
        print(t)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## 4. Simulador

 Rode o simulador do Turtlebot (use o Waffle).  Veja o guia em [../guides/simulador_ros.md](../guides/simulador_ros.md)
 
 Documente aqui as linhas necessárias para teleop e para abrir o Rviz
 
 Faça um screenshot do seu simulação em execução

## 5. Robô quadrado

Faça [este tutorial](../guides/projeto_rospython.md) de como criar um projeto Python que comanda o robô simulado.

Usando o simulador, crie um código que faça o robô fazer uma trajetória que aproxima um quadrado.

Baseie-se no código `roda.py`, construído durante o tutorial 

In [ ]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-

import rospy
from geometry_msgs.msg import Twist, Vector3
from math import pi


v = 0.3  # Velocidade linear
w = pi/3  # Velocidade angular
stop = 0
if __name__ == "__main__":
    rospy.init_node("roda_exemplo")
    pub = rospy.Publisher("cmd_vel", Twist, queue_size=3)

    try:
        while not rospy.is_shutdown():

            vel_forw = Twist(Vector3(v,0,0), Vector3(0,0,0))
            vel_turn = Twist(Vector3(0,0,0), Vector3(0,0,w))
            vel_stop = Twist(Vector3(0,0,0), Vector3(0,0,0))



            if stop == 0:
                pub.publish(vel_stop)
                rospy.sleep(2)

            while stop <= 4:
                pub.publish(vel_stop)
                rospy.sleep(2)

                pub.publish(vel_forw)
                rospy.sleep(1.8)


                pub.publish(vel_stop)
                rospy.sleep(2)


                pub.publish(vel_turn)
                rospy.sleep(1.5)

                pub.publish(vel_stop)
                rospy.sleep(2)

                stop += 1

    except rospy.ROSInterruptException:
        print("Ocorreu uma exceção com o rospy")


## 6. Robô indeciso

Usando o simulador e o LIDAR simulado, faça um robô avançar quando o obstáculo bem à sua frente estiver a menos de 1.0m e recuar quando estiver a mais de 1.02 m.

Baseie-se no código `le_scan.py` e `roda.py`, desenvolvidos [durante o tutorial](../guides/projeto_rospython.md)

In [ ]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-

import rospy
import numpy as np

from geometry_msgs.msg import Twist, Vector3
from sensor_msgs.msg import LaserScan


def scaneou(dado):
    print("Faixa valida: ", dado.range_min , " - ", dado.range_max )
    print("Leituras:")
    print(np.array(dado.ranges).round(decimals=2))
    dados = dado
    global dados
    #print("Intensities")
    #print(np.array(dado.intensities).round(decimals=2))

    


if __name__=="__main__":

    rospy.init_node("le_scan")

    velocidade_saida = rospy.Publisher("/cmd_vel", Twist, queue_size = 3 )
    recebe_scan = rospy.Subscriber("/scan", LaserScan, scaneou)



    while not rospy.is_shutdown():
        print("Oeee")
        vel_forw = Twist(Vector3(0.1,0,0), Vector3(0,0,0))
        vel_back = Twist(Vector3(-0.1,0,0), Vector3(0,0,0))
        # velocidade = Twist(Vector3(0, 0, 0), Vector3(0, 0, 1))
        velocidade_saida.publish(vel_forw)
        rospy.sleep(2)

        if dados.ranges[0] < 1:
            velocidade_saida.publish(vel_back)
            rospy.sleep(2)
        if dados.ranges[0] > 1.02:
            velocidade_saida.publish(vel_forw)
            rospy.sleep(2)
